In [1]:
import numpy as np
from numpy.testing import assert_allclose

In [2]:
# 3.2.4 シグモイド関数の実装
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [11]:
# 3.4.2 各層における信号伝達の実装
# 第1層目
X = np.array([1.0, 0.5])
W1 = np.array([[0.1, 0.3, 0.5], [0.2, 0.4, 0.6]])
B1 = np.array([0.1, 0.2, 0.3])

assert W1.shape ==(2, 3)
assert X.shape ==(2,)
assert B1.shape ==(3,)

A1 = np.dot(X, W1) + B1
assert_allclose(A1, np.array([0.3, 0.7, 1.1]))

Z1 = sigmoid(A1)
assert_allclose(Z1, np.array([0.57444252, 0.66818777, 0.75026011]))


In [16]:
# 第2層目
W2 = np.array([[0.1, 0.4], [0.2, 0.5], [0.3, 0.6]])
B2 = np.array([0.1, 0.2])

assert Z1.shape ==(3,)
assert W2.shape ==(3, 2)
assert B2.shape ==(2,)

A2 = np.dot(Z1, W2) + B2
Z2 = sigmoid(A2)

assert_allclose(A2, np.array([0.51615984, 1.21402696]))
assert_allclose(Z2, np.array([0.62624937, 0.7710107]))


In [21]:
# 出力層
def identity_function(x):
    return x

W3 = np.array([[0.1, 0.3], [0.2, 0.4]])
B3 = np.array([0.1, 0.2])

A3 = np.dot(Z2, W3) + B3
Y = identity_function(A3)

assert_allclose(A3, np.array([0.31682708, 0.69627909]))
assert_allclose(Y, np.array([0.31682708, 0.69627909]))


In [30]:
# 3.4.3　実装のまとめ
def init_network():
    network = {
        'W1' : np.array([[0.1, 0.3, 0.5], [0.2, 0.4, 0.6]]),
        'b1' : np.array([0.1, 0.2, 0.3]),
        'W2' : np.array([[0.1, 0.4], [0.2, 0.5], [0.3, 0.6]]),
        'b2' : np.array([0.1, 0.2]),
        'W3' : np.array([[0.1, 0.3], [0.2, 0.4]]),
        'b3' : np.array([0.1, 0.2]),
    }
    return network

def forward(network, x):
    W1, W2, W3, = network['W1'], network['W2'], network['W3']
    b1, b2, b3, = network['b1'], network['b2'], network['b3']

    a1 = np.dot(x, W1) + b1
    z1 = sigmoid(a1)
    a2 = np.dot(z1, W2) + b2
    z2 = sigmoid(a2)
    a3 = np.dot(z2, W3) + b3
    y = identity_function(a3)
    return y

network = init_network()
x = np.array([1.0, 0.5])
y = forward(network, x)

assert_allclose(y, np.array([0.31682708, 0.69627909]))


In [44]:
# 3.5.1 恒等関数とソフトマックス関数
a = np.array([0.3, 2.9, 4.0])
exp_a = np.exp(a) # 指数関数
assert_allclose(exp_a, np.array([1.34985881, 18.17414537, 54.59815003]))

sum_exp_a = np.sum(exp_a) # 指数関数の和
assert sum_exp_a == 74.1221542101633

y = exp_a / sum_exp_a
assert_allclose(y, np.array([0.018211273295547534, 0.2451918129350739, 0.7365969137693785]))


In [45]:
def softmax(a):
    exp_a = np.exp(a)
    sum_exp_a = np.sum(exp_a)
    y = exp_a / sum_exp_a
    return y

In [49]:
# 3.5.2 ソフトマックス関数の実装上の注意
# 大きすぎる値になるためnp.nanが返ってくる
assert_allclose(softmax(np.array([1010, 1000, 990])), np.array([np.nan, np.nan, np.nan]))


In [54]:
def softmax(a):
    c = np.max(a)
    exp_a = np.exp(a - c)
    sum_exp_a = np.sum(exp_a)
    y = exp_a / sum_exp_a
    return y
# 入寮信号の最大値を引くことで正しく計算ができている
assert_allclose(softmax(np.array([1010, 1000, 990])), np.array([9.99954600e-01, 4.53978686e-05, 2.06106005e-09]))

In [59]:
# ソフトマックス関数の特徴
y = softmax(np.array([0.3, 2.9, 4.0]))
assert_allclose(y, np.array([0.01821127329554753, 0.24519181293507392, 0.7365969137693786]))

# 総和が1になるため、出力を確率と解釈可能
assert np.sum(y) == 1.0
